##### 问题:
我们想定义一个类作为接口或者是抽象基类，这样可以在此之上执行类型检查并确保
在子类中实现特定的方法。

##### 解决方案:
要定义一个抽象基类，可以使用 abc 模块。示例如下：

In [29]:
from abc import ABCMeta, abstractmethod
class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxbytes=-1):
        pass
    @abstractmethod
    def write(self, data):
        pass 

抽象基类的核心特征就是不能被直接实例化。例如，如果尝试这么做，会得到错误提示：

In [30]:
#a = IStream()

相反，抽象基类是用来给其他的类当做基类使用的，这些子类需要实现基类中要求的
那些方法。示例如下：

In [31]:
class SocketStream(IStream):
    def read(self, maxbytes=-1):
        pass
    def write(self, data): 
        pass


抽象基类的主要用途是强制规定所需的编程接口。例如，一种看待 IStream 基类的方式
就是在高层次上指定一个接口规范，使其允许读取和写入数据。显式检查这个接口的
代码可以写成如下形式：

In [32]:
def serialize(obj, stream):
    if not isinstance(stream, IStream):
        raise TypeError('Expected an IStream') 


我们可能会认为这种形式的类型检查只有在子类化抽象基类（ABC）时才能工作，但是抽象基类也允许其他的类向其注册，然后实现所需的接口。例如，我们可以这样做：

In [33]:
import io
# Register the built-in I/O classes as supporting our interface
IStream.register(io.IOBase)
# Open a normal file and type check
f = open('foo.txt')
isinstance(f, IStream) # Returns True

True

应该提到的是，@abstractmethod 同样可以施加到静态方法、类方法和 property 属性上。
只要确保以合适的顺序进行添加即可，这里@abstractmethod 要紧挨着函数定义。示例
如下：

In [34]:
from abc import ABCMeta, abstractmethod
class A(metaclass=ABCMeta):
    @property
    @abstractmethod
    def name(self):
        pass
    @name.setter
    @abstractmethod
    def name(self, value):
        pass
    @classmethod
    @abstractmethod
    def method1(cls):
        pass
    @staticmethod
    @abstractmethod
    def method2():
        pass

标准库中已经预定义好了一些抽象基类。collections 模块中定义了多个和容器还有迭代
器（序列、映射、集合等）相关的抽象基类。numbers 库中定义了和数值对象（整数、
浮点数、复数等）相关的抽象基类。io 库中定义了和 I/O 处理相关的抽象基类。
可以使用这些预定义好的抽象基类来执行更加一般化的类型检查。下面是一些例子：

In [35]:
import collections
x=[]
# Check if x is a sequence
if isinstance(x, collections.Sequence):
    pass
# Check if x is iterable
if isinstance(x, collections.Iterable):
    pass
# Check if x has a size
if isinstance(x, collections.Sized):
    pass
# Check if x is a mapping
if isinstance(x, collections.Mapping):
    pass

应该提到的是，在写作本节时，某些库和模块并没有像我们所期望的那样利用预定义
好的抽象基类。例如：

In [36]:
from decimal import Decimal
import numbers
x = Decimal('3.4')
isinstance(x, numbers.Real) # Returns False

False

虽然从技术上说 3.4 是一个实数，由于我们无意中将浮点数和小数混在一起，这里的类
型检查没有起到应有的作用。因此，如果使用了抽象基类的功能，明智的做法是仔细
编写测试用例来验证其行为是否是所期待的。

尽管抽象基类使得类型检查变得更容易了，但不应该在程序中过度使用它。Python 的
核心在于它是一种动态语言，它带来了极大的灵活性。如果处处都强制实行类型约束，
则会使得代码变得更加复杂，而这本不应该如此。我们应该拥抱 Python 的灵活性。